In [0]:
!pip install tensorflow==1.14.0

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from PIL import Image

In [0]:
from google.colab import drive #STU NAPERVILLE ACCOUNTS DON'T WORK - USE YOUR PERSONAL GOOGLE ACCOUNTS
drive.mount('/content/drive')

In [21]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=[0.5,1.5], rotation_range=5)
#replace "dataset1" with dataset you want to test
train_generator = train_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/combined/training",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255, brightness_range=[0.5,1.5], rotation_range=5)
test_generator = test_datagen.flow_from_directory(
        "/content/drive/My Drive/ROADREADER DATASETS/combined/tests",
        target_size=(300, 400),
        batch_size=32, 
        color_mode="rgb",
        class_mode="sparse",
        shuffle=True)

Found 5717 images belonging to 2 classes.
Found 1365 images belonging to 2 classes.


In [22]:
model = keras.Sequential([
  layers.Conv2D(14, 3, activation='relu', input_shape=(300,400,3)),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(28, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(42, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(84, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Conv2D(168, 3, activation='relu'),
  layers.MaxPooling2D((2,2), padding='same'),
  layers.Flatten(),
  layers.Dense(2000, activation='relu'),
  layers.Dense(200, activation='relu'),
  layers.Dense(2, activation='softmax'), #classification layer; 0 for none 1 for stop
])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 298, 398, 14)      392       
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 149, 199, 14)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 147, 197, 28)      3556      
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 74, 99, 28)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 72, 97, 42)        10626     
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 36, 49, 42)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 34, 47, 84)       

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
#model.fit(x=train_images,y=train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=128)

model.fit_generator(
        train_generator,
        steps_per_epoch=230,
        epochs=25,
        validation_data=test_generator,
        validation_steps=55,
        workers=6,
        use_multiprocessing=True
        #max_queue_size=10
        )

'''
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)
                    '''

model.save_weights('/content/model.h5')

Epoch 1/25
230/230 [==============================] - 157s 681ms/step - loss: 0.3190 - acc: 0.8422 - val_loss: 0.1118 - val_acc: 0.9657
Epoch 2/25
230/230 [==============================] - 153s 665ms/step - loss: 0.0867 - acc: 0.9718 - val_loss: 0.0481 - val_acc: 0.9823
Epoch 3/25
230/230 [==============================] - 155s 674ms/step - loss: 0.0613 - acc: 0.9796 - val_loss: 0.0508 - val_acc: 0.9817
Epoch 4/25
230/230 [==============================] - 159s 689ms/step - loss: 0.0447 - acc: 0.9860 - val_loss: 0.0249 - val_acc: 0.9920
Epoch 5/25
230/230 [==============================] - 155s 673ms/step - loss: 0.0458 - acc: 0.9853 - val_loss: 0.0347 - val_acc: 0.9874
Epoch 6/25
230/230 [==============================] - 155s 674ms/step - loss: 0.0324 - acc: 0.9899 - val_loss: 0.0181 - val_acc: 0.9937
Epoch 7/25
230/230 [==============================] - 156s 676ms/step - loss: 0.0269 - acc: 0.9893 - val_loss: 0.0276 - val_acc: 0.9897
Epoch 8/25
230/230 [============================